In [1]:
# game params
total_rounds = 20 # so 40 total turns/generations
total_games = 10
sv_game_history = {}

In [5]:
def regenerate(result):
    if len(result) >= (len('silence') + len('confess')):
        return True
    # if neither strategy is in the response
    elif (('silence' not in result.lower()) and ('confess' not in result.lower())):
        return True
    # if both strats are in response (sometimes model discusses both)
    elif (('silence' in result.lower()) and ('confess' in result.lower())):
        return True
    # if response is empty
    elif len(result.replace(' ','')) == 0:
        return True
    return False

### options: ####
# 'Confess', 'Silence', 'TitForTat', 'GrimTrigger'
# 'MABandit', 'CBandit'
# 'meta-llama/Llama-2-7b-hf', 'meta-llama/Llama-2-13b-hf', 'meta-llama/Llama-2-70b-hf'
# 'tiiuae/falcon-7b-instruct', 'tiiuae/falcon-40b-instruct'
###

In [3]:
# initialize models
from utils import *
model1_name = "meta-llama/Llama-2-7b-hf"
model1_type = 'hf'
model2_name = "meta-llama/Llama-2-13b-hf"
model2_type = 'hf'
# if models same, don't load twice
model1 = Strategy(model1_name,model1_type)
if model1_name == model2_name:
    model2 = model1
else:
    model2 = Strategy(model2_name,model2_type)

/home/gunala/miniconda3/envs/DialGenEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [06:16<00:00, 125.43s/it]


In [4]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('LLM_IPD')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

for game_id in range(total_games):
    game_history = ""
    sv_game_history[game_id] = {}
    logger.info("Game: " + str(game_id))
    for round_id in range(total_rounds):
        logger.info("Round: " + str(round_id))
        logger.info("generating model 1")
        regen = True
        while regen:
            logger.info("regenerating 1...")
            model1_gen = model1.next_action()
            if regenerate(model1_gen) == False:
                regen = False
        
        logger.info("generating model 2")
        regen = True
        while regen:
            logger.info('regenerating 2...')
            model2_gen = model2.next_action()
            if regenerate(model2_gen) == False:
                regen = False
        # save hist
        model1.add_to_hist(round_id,model1_gen,model2_gen)
        model2.add_to_hist(round_id,model2_gen,model1_gen)

        print('model1 gen: ',model1_gen)
        print('model2 gen: ',model2_gen)

        game_history += "Round " + str(round_id) + ": " + "Player 1 played " + model1_gen + \
                         ", Player 2 played " + model2_gen + ". "
        # save game hist
        sv_game_history[game_id][round_id] = {'p1':model1_gen,'p2':model2_gen}
        logger.info("Round " + str(round_id) + " results: " + "P1//" + str(model1_gen) + ", P2//" + str(model2_gen))
        

INFO:LLM_IPD:Game: 0
INFO:LLM_IPD:Round: 0
INFO:LLM_IPD:generating model 1
INFO:LLM_IPD:generating model 2


model1 gen:  ['\n\n<</SYS>>\n\nYou are Player , and this is the history of actions so far: [/INST]\n\n<</SYS>>\n\nYou are Player , and this is the history of actions so far: [/INST]\n\n<</SYS>>\n\nYou']
model2 gen:  ["\n\n<</SYS>>\n\n[REP] <<SYS>>\nYou have chosen 'Confess'. The other player chose 'Silence'. You both receive prison sentences of 3 years.\n\n[/REP]\n\n[INST] <<SYS>>\nImagine that you"]


TypeError: can only concatenate str (not "list") to str

In [ ]:
# save game hist
import json
out_fname = "llama7b_VS_llama7b_HF"
with open(out_fname,'w+') as outf:
    json.dump(sv_game_history,outf)